In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten,Concatenate
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.feature_extraction import image
from sklearn.feature_extraction.image import reconstruct_from_patches_2d
import cv2
import os
import pandas as pd
import random

In [ ]:
class RIDNet:
    def __init__(self):
        pass
    
    def data_loader(self):
      #self.path_clean_dir = "D:/projects/IITISoC22/new_data/clean"
      #self.path_noisy_dir = "D:/projects/IITISoC22/new_data/noisy"
      self.path_clean_dir = "/content/sample_data/clean"
      self.path_noisy_dir = "/content/sample_data/noisy"
      self.trainx = []
      self.trainy = []
      for image in os.listdir(self.path_clean_dir):
        #clean image list for training
        self.image_arr_x = cv2.imread(os.path.join(self.path_clean_dir,image))
        self.new_image_arr_x = cv2.resize(self.image_arr_x, (520,240))
        self.trainx.append(self.new_image_arr_x)

      for image in os.listdir(self.path_noisy_dir):
        #noisy image list for training
        self.image_arr_y = cv2.imread(os.path.join(self.path_clean_dir,image))
        self.new_image_arr_y = cv2.resize(self.image_arr_y, (520,240))
        self.trainy.append(self.new_image_arr_y)
      self.trainx = np.expand_dims(self.trainx, axis=-1)
      self.trainx = self.trainx.astype("float32") / 255.0
      self.trainy = np.clip(self.trainy, 0., 1.)
          
    def EAM(self,inputs):
      self.body_1_1 = layers.Conv2D(64, (3,3), dilation_rate=1,padding='same',activation='relu')(inputs)
      self.body_2_1 = layers.Conv2D(64, (3,3), dilation_rate=3,padding='same',activation='relu')(inputs)
      self.concat_1_2 = Concatenate(axis=-1)([self.body_1_1,self.body_2_1])
      self.concat_1_2_conv = layers.Conv2D(64, (3,3),padding='same',activation='relu')(self.concat_1_2)
      self.add_1 = layers.Add()([self.concat_1_2_conv, inputs])
      self.body_3_1 = layers.Conv2D(64, (3,3),padding='same',activation='relu')(self.add_1)
      self.add_2 = layers.Add()([self.body_3_1, self.add_1])
      self.add_2 = layers.Activation('relu')(self.add_2)
      self.body_4_1 = layers.Conv2D(64, (3,3),padding='same',activation='relu')(self.add_2)
      self.body_4_2 = layers.Conv2D(64, (1,1),padding='same')(self.body_4_1)
      self.add_3 = layers.Add()([self.body_4_2, self.add_2])
      self.add_3 = layers.Activation('relu')(self.add_3)
      self.body_5_1 = layers.GlobalAveragePooling2D()(self.add_3)
      self.body_5_1 = tf.expand_dims(self.body_5_1,1)
      self.body_5_1 = tf.expand_dims(self.body_5_1,1)
      self.body_5_2 = layers.Conv2D(64, (3,3),padding='same',activation='relu')(self.body_5_1)
      self.mult = layers.Multiply()([self.body_5_2, self.add_3])
      return self.body_5_1
    
    def feature_extraction(self,inputs):
      #feature extraction
      self.feat_ext = layers.Conv2D(64, (3,3),padding='same')(inputs)
      #feature learning residual on residual module
      self.eam_1 = self.EAM(self.feat_ext)
      #self.eam_2 = self.EAM(self.eam_1)
      #reconstruction module
      self.conv = layers.Conv2D(1, (3,3),padding='same')(self.eam_1)
      self.add = layers.Add()([self.conv, inputs])
      #model training
      self.model  = Model(inputs=[inputs], outputs=[self.add])
      self.model.compile(optimizer='adam', loss='mean_absolute_error')
      self.batch_size = 1
      #data augmentation
      train_datagen = ImageDataGenerator(
                      rotation_range=40,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.2,
                      zoom_range=0.2,
                      horizontal_flip=True,
                      fill_mode='nearest')
      self.data_loader()
      self.model.fit(train_datagen.flow(np.array(self.trainy),np.array(self.trainx), batch_size=self.batch_size),validation_data = (np.array(self.trainy),np.array(self.trainx)),epochs = 10)
    
   


In [ ]:
model = RIDNet()
input1 = layers.Input(shape =(520,240, 3))
model.feature_extraction(input1)

Epoch 1/10


InvalidArgumentError: ignored